# Auto-atenção

In [2]:
from keras.layers import Input, Dense, Activation, TimeDistributed, Softmax, MultiHeadAttention, TextVectorization, Reshape, RepeatVector, Conv1D, Bidirectional, AveragePooling1D, UpSampling1D, Embedding, Concatenate, GlobalAveragePooling1D, LSTM, Multiply
from keras.models import Model
import tensorflow as tf
import keras
import numpy as np

# Ler um dataset e fazer batches
# DATASET_DIR = './datasets/frases/'
DATASET_DIR = './datasets/frases_classificacao/train/positive/'

from tensorflow.keras.utils import text_dataset_from_directory

dataset = text_dataset_from_directory(
    DATASET_DIR,
    labels=None,
    label_mode='int',
    class_names=None,
    batch_size=2048,
    max_length=None,
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    follow_links=False
)

from keras.layers import Input, TextVectorization
from keras.models import Model
vocab_size = 5000
seq_len = 10
vectorize_layer = TextVectorization(max_tokens=vocab_size, output_sequence_length=seq_len)
vectorize_layer.adapt(dataset)

Found 19937 files belonging to 1 classes.
Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-05-04 11:16:23.357845: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Já vimos que um dos maiores problemas na modelagem linguística é manter algum tipo de coerência temporal nos tokens que são gerados.

Um possível processo para modelar essa coerência temporal é o seguinte.

Começaremos com três representações projetadas à partir da nossa entrada:

$$
Q = XW_q \hspace{0.5in} V = XW_v \hspace{0.5in} K = XW_k
$$

Depois, combinamos da seguinte forma:

1. O produto interno $QK^T$ informa o quanto cada entrada, ao longo do tempo, depende das outras entradas,
1. Essa dependência é escalada pela dimensão da representação de $X$ para evitar a explosão do espaço latente
1. O resultado é ponderado por softmax, de forma que a soma das dependências ao longo do tempo é 1 e pode ser interpretada como uma probabilidade
1. O resultado disso tudo pondera as representações $V$:

$$
S = D(Q, K, V) = \text{softmax}\begin{pmatrix} \frac{QK^T}{\sqrt{d_q}} \end{pmatrix}V
$$

Esse processo pode ser feito em várias etapas paralelas que são somadas em uma mesma camada num processo chamado de *multi head*.

## Exercício 1
**Objetivo: analisar o processo de multi-head attention no Keras**

Analisando o código abaixo, verifique:

Quais são as entradas e saídas de um layer multi-head attention? O que cada dimensão significa?


In [3]:
seq_len = 10
vocab_size = 5000
def predict_word(seq_len, latent_dim, vocab_size):
    input_layer = Input(shape=(seq_len-1,))
    x = input_layer
    x = Embedding(vocab_size, latent_dim, name='embedding', mask_zero=True)(x)
    x = MultiHeadAttention(num_heads=3, key_dim=2)(x, value=x)
    x = GlobalAveragePooling1D()(x)
    latent_rep = x
    x = Dense(vocab_size)(x)
    x = Softmax()(x)
    return Model(input_layer, x), Model(input_layer, latent_rep)

predictor, latent = predict_word(seq_len, 15, vocab_size)
predictor.summary()
#opt = keras.optimizers.SGD(learning_rate=1, momentum=0.9)
opt = keras.optimizers.Nadam(learning_rate=0.1)
loss_fn = keras.losses.SparseCategoricalCrossentropy(
    ignore_class=1,
    name="sparse_categorical_crossentropy",
)

predictor.compile(loss=loss_fn, optimizer=opt, metrics=["accuracy"])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 9)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 9, 15)        75000       ['input_1[0][0]']                
                                                                                                  
 multi_head_attention (MultiHea  (None, 9, 15)       393         ['embedding[0][0]',              
 dAttention)                                                      'embedding[0][0]']              
                                                                                                  
 global_average_pooling1d (Glob  (None, 15)          0           ['multi_head_attention[0][0]'

## Exercício 2
**Objetivo: treinar e usar um modelo linguístico com multi-head attention**

Usando o código abaixo, faça o treinamento de um modelo linguístico que usa multi-head attention. 

Após, use as funções que você já fez nas aulas anteriores para usar o modelo para gerar texto.

In [4]:
def separar_ultimo_token(x):
    x_ = vectorize_layer(x)
    x_ = x_[:,:-1]
    y_ = x_[:,-1:]
    return x_, y_

history = predictor.fit(dataset.map(separar_ultimo_token), epochs=40, verbose=1)

Epoch 1/40
10/10 [==============================] - 4s 167ms/step - loss: 7.3151 - accuracy: 0.0272
Epoch 2/40
10/10 [==============================] - 2s 131ms/step - loss: 6.0981 - accuracy: 0.0498
Epoch 3/40
10/10 [==============================] - 2s 117ms/step - loss: 6.0522 - accuracy: 0.0495
Epoch 4/40
10/10 [==============================] - 2s 117ms/step - loss: 5.8161 - accuracy: 0.0613
Epoch 5/40
10/10 [==============================] - 1s 115ms/step - loss: 5.4498 - accuracy: 0.0798
Epoch 6/40
10/10 [==============================] - 2s 124ms/step - loss: 5.2334 - accuracy: 0.0956
Epoch 7/40
10/10 [==============================] - 2s 123ms/step - loss: 4.9742 - accuracy: 0.1080
Epoch 8/40
10/10 [==============================] - 2s 123ms/step - loss: 4.7425 - accuracy: 0.1253
Epoch 9/40
10/10 [==============================] - 1s 117ms/step - loss: 4.7957 - accuracy: 0.1183
Epoch 10/40
10/10 [==============================] - 2s 124ms/step - loss: 4.4145 - accuracy: 0.1456

## Exercício 3
**Objetivo: criar um classificador de texto com multi-head attention**

Usando a camada multi-head attention, projete e treine um classificador de texto para uma aplicação à sua escolha. Qual foi o accuracy que você obteve?

In [ ]:
def convolve_and_downsample(input_n_samples, input_embedding_size, n_filters, kernel_size=3, **kwargs):
    input_layer = Input(shape=(input_n_samples,input_embedding_size))
    x = input_layer
    x = Conv1D( filters=n_filters,
                kernel_size=kernel_size,
                padding='same',
                use_bias=False,
                )(x)
    x = AveragePooling1D(pool_size=2)(x)
    x = Activation('elu')(x)
    return Model(input_layer, x, **kwargs)

seq_len = 10
vocab_size = 5000
def predict_word(seq_len, latent_dim, vocab_size):
    input_layer = Input(shape=(seq_len-1,))
    x = input_layer
    x = Embedding(vocab_size, latent_dim, name='embedding', mask_zero=True)(x)
    x = MultiHeadAttention(num_heads=3, key_dim=2)(x, value=x)
    x = GlobalAveragePooling1D()(x)
    x = convolve_and_downsample(256, 2, number_of_ngrams, n_gram_size, name='ngramas')(x)
    latent_rep = x
    x = Dense(vocab_size)(x)
    x = Softmax()(x)
    return Model(input_layer, x), Model(input_layer, latent_rep)

predictor, latent = predict_word(seq_len, 15, vocab_size)
predictor.summary()
#opt = keras.optimizers.SGD(learning_rate=1, momentum=0.9)
opt = keras.optimizers.Nadam(learning_rate=0.1)
loss_fn = keras.losses.SparseCategoricalCrossentropy(
    ignore_class=1,
    name="sparse_categorical_crossentropy",
)

predictor.compile(loss=loss_fn, optimizer=opt, metrics=["accuracy"])

